In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path

# settings for making nice pdfs
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
base_folder_string = 'output_0824_tiling600_DSBStabilization_'
folders = [foldername for foldername in os.listdir('Data/') if foldername.startswith(base_folder_string)]

base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir = folders[m]
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    proc_list = list(set([stats_file[i].split('_proc')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_list_sorted = [int(a) for a in proc_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_supersep')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    dsb_numerical_sorted = [float(a) for a in dsb_list_sorted]
    supersep_list = list(set([stats_file[i].split('supersep')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    supersep_numerical_list = [int(float(a)) for a in supersep_list]
    supersep_list_sorted = [x for _,x in sorted(zip(supersep_numerical_list,supersep_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))
        ctcf_bs_success =  np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))
        ctcf_bs_time0 =   np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))

    for i in range(len(proc_list_sorted)):
        for j in range(len(dsb_list_sorted)):
            par_combo = 'sep' + sep_list_sorted[0] + '_proc' + proc_list_sorted[i] + '_superproc' + superproc_list_sorted[0] + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[j] + '_supersep' + supersep_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]           
            filename = 'Data/'+data_dir + '/stats_df_' + par_combo + '.csv'
            if path.exists(filename):
                stats_df = pd.read_csv(filename)
                stats_df = stats_df.drop(columns='Unnamed: 0')
                stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                if len(sep_proc_1st_pass)>0:
                    ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 *2
                else:
                    ctcf_bs_fp[m,i,j]=np.nan


                portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
                portion_df = portion_df.drop(columns='Unnamed: 0')
                portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]*100
                ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100

# Theory prediction from Mathematica Notebook DSBstabilization_20211004.nb, Prob
theory = np.asarray([[0.0484416, 0.0631982, 0.0725675, 0.0789825, 0.0836343, 0.0871565, \
0.0899138, 0.0921299, 0.0939495, 0.0954699, 0.0967592, 0.0978662, \
0.098827, 0.0996687, 0.100412, 0.101074, 0.101666, 0.102199, \
0.102682, 0.103121, 0.103523, 0.103891, 0.104229, 0.104542, 0.104832, \
0.105101, 0.105351, 0.105585, 0.105804, 0.106009, 0.106202],[0.113293, 0.1384, 0.153363, 0.163244, 0.170242, 0.175454, 0.179484, \
0.182693, 0.185308, 0.18748, 0.189313, 0.190879, 0.192234, 0.193417, \
0.194459, 0.195384, 0.196211, 0.196954, 0.197625, 0.198235, 0.198791, \
0.1993, 0.199768, 0.2002, 0.2006, 0.200971, 0.201316, 0.201638, \
0.201939, 0.202221, 0.202485],[0.204531, 0.235863, 0.253608, 0.264999, 0.272924, 0.278753, 0.283219, \
0.286751, 0.289613, 0.291979, 0.293968, 0.295663, 0.297126, 0.298399, \
0.299519, 0.300512, 0.301397, 0.302191, 0.302909, 0.303559, 0.304152, \
0.304694, 0.305193, 0.305652, 0.306076, 0.30647, 0.306836, 0.307178, \
0.307497, 0.307795, 0.308076]])*100

slist = np.asarray([1., 1.5, 2., 2.5, 3., 3.5, 4., 4.5, 5., 5.5, 6., 6.5, 7., 7.5, 8., \
8.5, 9., 9.5, 10., 10.5, 11., 11.5, 12., 12.5, 13., 13.5, 14., 14.5, \
15., 15.5, 16.])


colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))
    
for i in range(len(proc_list_sorted)):
    axs[0].errorbar(dsb_numerical_sorted, np.transpose(np.mean(ctcf_bs_success,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_success,axis=0)[i,:],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)
for i in range(len(proc_list_sorted)):
    x = proc_numerical_list_sorted[i]/sep_numerical_list[0] 
    y = (1+4/x)/(1+2/x)**2
    axs[0].plot(slist,theory[i],color= colors[i],linewidth=3,linestyle='dashed')
    
axs[0].set_ylim(0,100)    
axs[0].set_ylabel('Synapsis efficiency (%)')
axs[0].set_xlabel('DSB stabilization factor')
axs[0].set_xticks(dsb_numerical_sorted[0:])
axs[0].tick_params(direction='out', length=8, width=2)
for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)

# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    
for i in range(len(proc_list_sorted)):
    axs[1].errorbar(dsb_numerical_sorted , np.transpose(np.mean(ctcf_bs_fp,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_fp,axis=0)[i,:],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

axs[1].set_ylabel('Mean synapsis time (min)')
axs[1].set_xlabel('DSB stabilization factor')
axs[1].set_xticks(dsb_numerical_sorted[0:])
axs[1].tick_params(direction='out', length=8, width=2)
for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+proc for proc in proc_list_sorted]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')

plt.savefig('Figures/'+'DSB_stabilization.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()

In [ ]:
base_folder_string = 'output_0824_with residence_10 Mb apart_tiling600_2populationLEFs_GapMethod_DSBStabilization_'
folders = [foldername for foldername in os.listdir('Data/') if foldername.startswith(base_folder_string)]

base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir =folders[m]
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    proc_list = list(set([stats_file[i].split('_proc')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_list_sorted = [int(a) for a in proc_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_supersep')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    dsb_numerical_sorted = [float(a) for a in dsb_list_sorted]
    supersep_list = list(set([stats_file[i].split('supersep')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    supersep_numerical_list = [int(float(a)) for a in supersep_list]
    supersep_list_sorted = [x for _,x in sorted(zip(supersep_numerical_list,supersep_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))
        ctcf_bs_success =  np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))
        ctcf_bs_time0 =   np.zeros((len(folders),len(proc_list_sorted),len(dsb_list_sorted)))

    for i in range(len(proc_list_sorted)):
        for j in range(len(dsb_list_sorted)):
            par_combo = 'sep' + sep_list_sorted[0] + '_proc' + proc_list_sorted[i] + '_superproc' + superproc_list_sorted[0] + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[j] + '_supersep' + supersep_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]           
            filename = 'Data/'+data_dir + '/stats_df_' + par_combo + '.csv'
            if path.exists(filename):
                stats_df = pd.read_csv(filename)
                stats_df = stats_df.drop(columns='Unnamed: 0')
                stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                if len(sep_proc_1st_pass)>0:
                    ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 *2
                else:
                    ctcf_bs_fp[m,i,j]=np.nan


                portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
                portion_df = portion_df.drop(columns='Unnamed: 0')
                portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]*100
                ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100


# Theory prediction from Mathematica Notebook DSBstabilization_20211004.nb, Probconstrained
theory = np.asarray([[0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, \
0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, \
0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, \
0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, 0.707582, \
0.707582, 0.707582, 0.707582],[0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, \
0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, \
0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, \
0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, 0.822697, \
0.822697, 0.822697, 0.822697],[0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, \
0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, \
0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, \
0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, 0.892005, \
0.892005, 0.892005, 0.89200]])*100

slist = np.asarray([1., 1.5, 2., 2.5, 3., 3.5, 4., 4.5, 5., 5.5, 6., 6.5, 7., 7.5, 8., \
8.5, 9., 9.5, 10., 10.5, 11., 11.5, 12., 12.5, 13., 13.5, 14., 14.5, \
15., 15.5, 16.])


colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))
    
for i in range(len(proc_list_sorted)):
    axs[0].errorbar(dsb_numerical_sorted, np.transpose(np.mean(ctcf_bs_time0,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_time0,axis=0)[i,:],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)
for i in range(len(proc_list_sorted)):
    x = proc_numerical_list_sorted[i]/sep_numerical_list[0] 
    y = (1+4/x)/(1+2/x)**2
    axs[0].plot(slist,theory[i],color= colors[i],linewidth=3,linestyle='dashed')
    
axs[0].set_ylim(0,100)    
axs[0].set_ylabel('% DSB sites constrained')
axs[0].set_xlabel('DSB stabilization factor')
axs[0].set_xticks(dsb_numerical_sorted[0:])
axs[0].tick_params(direction='out', length=8, width=2)
for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)

# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    
for i in range(len(proc_list_sorted)):
    axs[1].errorbar(dsb_numerical_sorted , np.transpose(np.mean(ctcf_bs_fp,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_fp,axis=0)[i,:],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

axs[1].set_ylabel('Mean synapsis time (min)')
axs[1].set_xlabel('DSB stabilization factor')
axs[1].set_xticks(dsb_numerical_sorted[0:])
axs[1].tick_params(direction='out', length=8, width=2)
for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+proc for proc in proc_list_sorted]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')

plt.savefig('Figures/'+'DSB_stabilization_Pconstrained.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()